In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from hmm import HMM

In [2]:
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')

In [3]:
anime = anime.sort_values(by='anime_id')
anime = anime[anime['members'] > 10000]

In [4]:
# only consider slice of life anime
anime = anime[anime['genre'].str.contains('Slice of Life') == True]

In [5]:
# aggregrate people's votes -> counts of which users voted more
counts = rating.groupby(by='user_id').agg('count')
counts = counts.rename(index=str, columns={'anime_id': 'counts'})
del counts['rating']
counts = counts.reset_index()

In [6]:
# only consider anime rated >= 7
rating = rating[rating['rating'] >= 7]

# people who rated more than n anime
top_counts = counts[(counts['counts'] >= 80)]  # ~20k users

In [7]:
# filter only ratings from people who satisfy the above criteria
rating_ = rating[rating['user_id'].isin(top_counts['user_id'])]  # filtered rating database - include top users only
rating_.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
156,3,20,8


In [8]:
len(rating_['anime_id'])  # 2653526

TIME_STEP = 2000  # anime_ids between time steps. min = 17; max = 34525.

# need to group by anime_id
rating_ = rating_.assign(time=rating_['anime_id']/TIME_STEP)
rating_['time'] = rating_['time'].apply(lambda x: int(x))

In [9]:
# filtered rating database - include only non-filtered anime
rating_ = rating_[rating_['anime_id'].isin(anime['anime_id'])]

# randomly sample 1000 users
user_ids = rating_['user_id'].unique()
user_ids = np.random.choice(user_ids, 1000, replace=False)
animes = anime['anime_id'].unique()
anime_id_to_index = {}
for i in range(len(animes)):
    anime = animes[i]
    anime_id_to_index[anime] = i
    
# build an "observation sequence"
max_time = rating_['time'].max()
observation_seqs = np.zeros(shape=(len(user_ids), max_time), dtype=object)
for u in range(len(user_ids)):
    for t in range(max_time):  
        observation_seqs[u][t] = []
for u in tqdm(range(len(user_ids))):
    rating_per_user = rating_[rating_['user_id'] == user_ids[u]]
    for t in range(max_time):
        observation_seqs[u][t] = \
            rating_per_user[rating_per_user['time'] == t]['anime_id'].map(anime_id_to_index).tolist()
        if len(observation_seqs[u][t]) == 0:
             # fill in blanks with random anime -> TODO
            observation_seqs[u][t] = [anime_id_to_index[np.random.choice(animes)]] 

100%|██████████| 1000/1000 [00:46<00:00, 21.29it/s]


In [10]:
# hold-out time t anime
time_t_anime = []
for u in range(len(observation_seqs)):
    time_t_anime.append(observation_seqs[u][-1])

In [11]:
observation_seqs = np.resize(observation_seqs, (len(user_ids), max_time-1))

In [12]:
# average anime watched per time-step
total = 0
for seq in observation_seqs:
    total += sum([len(x) for x in seq])/len(seq)
total /= len(observation_seqs)
total

1.9946666666666697

In [14]:
n_states = 30
n_items = len(animes)
hmm = HMM(n_items, n_states, n_items + 5)

hmm.baum_welch(observation_seqs)


 35%|███▌      | 7/20 [21:31<39:58, 184.49s/it]

AssertionError: 

In [15]:
hmm.A

array([[  3.38349331e-02,   3.30723493e-02,   3.31059576e-02,
          3.30668314e-02,   3.30668314e-02,   1.37087664e+19,
          3.30668314e-02,   3.30668504e-02,   5.25286200e+25,
          3.30668314e-02,   3.30668314e-02,   3.30668314e-02,
          3.30668329e-02,   3.30668421e-02,   3.30670487e-02,
          3.30668493e-02,   3.30668314e-02,   3.30668318e-02,
          3.31078087e-02,   3.30668314e-02,   3.30680318e-02,
          4.21451197e-02,   3.77609911e-02,   3.30931528e-02,
          3.30668314e-02,   3.30668527e-02,   3.30668314e-02,
          4.94787108e+07,   3.35192971e-02,   3.30668320e-02],
       [  3.32214560e-02,   3.33785533e-02,   3.32110687e-02,
          3.32109380e-02,   3.32109380e-02,   2.45168343e+27,
          3.32184300e-02,   3.32109380e-02,   5.48061349e+31,
          3.32109380e-02,   3.32109380e-02,   3.32109380e-02,
          3.32109380e-02,   3.57321108e-02,   3.32109382e-02,
          3.32109380e-02,   3.32109380e-02,   3.32109380e-02,
       